In [1]:
pip install django
pip install django-rest-framework
pip install django-cors-headers
pip install django-extensions

  Using cached django_extensions-3.1.1-py3-none-any.whl (222 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Hvordan fordele eiendeler med utgangspunkt i ønskelister?

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import cvxopt

## Maksimal flyt med Edmonds Karp

In [14]:
users = ['A', 'B', 'C']
assets = ['a', 'b', 'c', 'd', 'e']

wishlists = [np.random.permutation(len(assets)) for u in users]
wishlists

[array([0, 4, 2, 1, 3]), array([1, 3, 4, 0, 2]), array([3, 4, 0, 1, 2])]

In [32]:
class Edge():
    name = ""
    flows = {}
    capacities = {}
    
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return self.name + ": " + str(self.edges)
    
user_nodes = [Node(u) for u in users]
asset_nodes = [Node(a) for a in assets]
s = Node("s")
t = Node("t")

In [35]:
for i in range(len(users)):
    user_nodes[i].edges = {asset_nodes[j]: [0, wishlists[i][j]] for j in range(len(assets))} | {s: [0, len(assets)]}
    
for i in range(len(assets)):
    asset_nodes[i].edges = {user_nodes[j]: [0, wishlists[j][i]) for j in range(len(users))} | {t: (0, len(assets))}
    
s.edges = {u_n: (0 len(assets)) for u_n in user_nodes}
t.edges = {a_n: len(assets) for a_n in asset_nodes}

In [48]:
flows = {(u_n, a_n): 0 for u_n, a_n in itertools.product(user_nodes, asset_nodes)}\
        | {(s, u_n): 0 for u_n in user_nodes}\
        | {(a_n, t): 0 for a_n in asset_nodes}

In [51]:
edges = list(flows.keys())

In [ ]:
flow = 0
while True:
    queue = [s]
    visited = []
    while len(q) > 0:
        current = queue.pop()
        for edge in current.edges:
            

Etter litt refleksjon har jeg kommet på at løsninger på maksimal flyt kan gi fordelinger der enkelteiendeler må deles mellom brukere. Dette er upraktisk i praksis. Vi kan derfor ikke bruke maksimal flyt.

## 0-1 Integer Lineær programmering
"Minimize c*x subject to Ax <=b, x ∈ {0, 1}"

In [22]:
relation_weights = {
    'child': 1,
    'parent': 0.8,
    'sibling': 0.7,
    'grandchildren': 0.6,
    'grandparent': 0.4,
    'pibling': 0.15,
    'other': 0.1
}

relation_priority = ['child', 'parent', 'sibling', 'grandchildren', 'grandparent', 'pibling', 'other']

In [23]:
users = ['A', 'B', 'C']
relations = np.random.choice(relation_priority, len(users))
assets = ['a', 'b', 'c', 'd', 'e']

wishlists = [np.random.permutation(len(assets)) for u in users]
wish_hstack = np.hstack(wishlists) - len(assets)
c = np.array([wish_hstack[i] * relation_weights[relations[i//len(assets)]] for i in range(len(wish_hstack))])
c

array([-1.4, -0.7, -2.8, -2.1, -3.5, -3.5, -2.1, -2.8, -1.4, -0.7, -2.4,
       -3. , -0.6, -1.2, -1.8])

In [24]:
relations

array(['sibling', 'sibling', 'grandchildren'], dtype='<U13')

In [46]:
weight_factor = 1 / max(relation_weights[relation] for relation in relations)
weight_factor

1.4285714285714286

In [25]:
max_one_user_per_asset = [[1 if (i-j)%len(assets)==0 else 0 for i in range(len(c))] for j in range(len(assets))]
fair_distribution = [[wish_hstack[i] if len(assets)*j <= i < len(assets)*(j+1) else 0 for i in range(len(c))] for j in range(len(users))]
A = np.array(max_one_user_per_asset + fair_distribution)
A

array([[ 1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1],
       [-2, -1, -4, -3, -5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, -5, -3, -4, -2, -1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -4, -5, -1, -2, -3]])

In [47]:
max_one_user_per_asset_sum = [1] * len(assets)
fair_distribution_sum = [wish_hstack[:len(assets)].sum() * relation_weights[relation] * weight_factor / len(users) for relation in relations]
b = np.array(max_one_user_per_asset_sum + fair_distribution_sum)
b

array([ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
       -5.        , -5.        , -4.28571429])

In [48]:
A.shape

(8, 15)

In [49]:
G = cvxopt.matrix(A, tc='d')
G

<8x15 matrix, tc='d'>

In [50]:
h=cvxopt.matrix(b, tc='d')
h

<8x1 matrix, tc='d'>

In [51]:
from cvxopt.glpk import ilp

solution = ilp(c=cvxopt.matrix(c, tc='d'),
               G=cvxopt.matrix(A, tc='d'),
               h=cvxopt.matrix(b, tc='d'),
               B=set(range(len(c))))
solution

('optimal', <15x1 matrix, tc='d'>)

In [52]:
opt = np.array(list(solution[1]))
opt

array([0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

In [53]:
np.matmul(A, opt)

array([  1.,   1.,   1.,   1.,   1., -12.,  -5.,  -5.])

Litt testing og logikk tilsier at denne algoritmen er optimal og rettferdig. Vi går derfor for binær linear programmering som løsning.

## Testing av full_distribution()

In [3]:
import django
django.setup()

In [4]:
from roddi.models import *

In [5]:
Asset.objects.all()

<QuerySet [<Asset: Vase>, <Asset: Maleri>, <Asset: Gevær>, <Asset: Bord>, <Asset: Stol>, <Asset: PC>]>

In [6]:
Asset.objects.all().delete()
User.objects.all().delete()
Estate.objects.all().delete()
Comment.objects.all().delete()
Wish.objects.all().delete()

(0, {})

In [7]:
users = [
    User.objects.create(name='Daniel', email='daniefs@stud.ntnu.no', age=20, relation_to_dead='pibling'),
    User.objects.create(name='Philip', email='philiped@stud.ntnu.no', age=20, relation_to_dead='sibling'),
    User.objects.create(name='Steffen', email='steffeah@stud.ntnu.no', age=20, relation_to_dead='parent')
]

for user in users:
    user.save()

In [8]:
assets = [
    Asset.objects.create(name='Vase', category='interiør'),
    Asset.objects.create(name='Maleri', category='kunst'),
    Asset.objects.create(name='Gevær', category='våpen'),
    Asset.objects.create(name='Bord', category='møbler'),
    Asset.objects.create(name='Stol', category='møbler'),
    Asset.objects.create(name='PC', category='elektronikk')
]

for asset in assets:
    asset.save()

In [9]:
Asset.objects.all()

<QuerySet [<Asset: Vase>, <Asset: Maleri>, <Asset: Gevær>, <Asset: Bord>, <Asset: Stol>, <Asset: PC>]>

In [10]:
estate = Estate.objects.create(name='Hans Hansens dødsbo', is_complete=False)
estate.save()
estate.users.add(*users)
estate.assets.add(*assets)

In [12]:
wishes = [
    [
        Wish.objects.create(user=user, asset=assets[i], priority=priorities[i])
        for i in range(len(assets))
        if (priorities := np.random.permutation(len(assets)) + 1) is not None
    ]
    for user in users
]

for wishlist in wishes:
    for wish in wishlist:
        wish.save()

In [14]:
estate.full_distribution()

In [15]:
[user.relation_to_dead for user in users]

['pibling', 'sibling', 'parent']

In [16]:
[user.get_ordered_wishlist() for user in users]

[[<Asset: Maleri>,
  <Asset: Gevær>,
  <Asset: Stol>,
  <Asset: Vase>,
  <Asset: Bord>,
  <Asset: PC>],
 [<Asset: Gevær>,
  <Asset: Vase>,
  <Asset: Maleri>,
  <Asset: Bord>,
  <Asset: PC>,
  <Asset: Stol>],
 [<Asset: PC>,
  <Asset: Stol>,
  <Asset: Maleri>,
  <Asset: Bord>,
  <Asset: Vase>,
  <Asset: Gevær>]]

In [19]:
[user.obtained_assets.all() for user in users]

[<QuerySet [<Asset: Bord>]>,
 <QuerySet [<Asset: Vase>, <Asset: Maleri>]>,
 <QuerySet [<Asset: Gevær>, <Asset: Stol>, <Asset: PC>]>]

In [16]:
estate.is_complete

True

Alt ser ut til å fungere

In [20]:
?ilp